In [1]:
%matplotlib
from biorefineries import oilcane as oc
import biosteam as bst
from biosteam.plots import sankey
from biosteam.utils import colors, CABBI_colors
from colorpalette import ColorWheel, Color
from matplotlib import cm
import numpy as np
bst.nbtutorial()

other = CABBI_colors.grey.tint(20)
orange_RGB = CABBI_colors.orange.shade(20).RGB
oil = Color(fg=0.70 * orange_RGB + 0.30 * other.RGB)
# water = Color(fg=0.70 * CABBI_colors.blue_light.tint(10).RGB + 0.30 * other.RGB)

class NodeHandle(sankey.Handle):
    __slots__ = ('node_colors',)

    def _init(self, node_colors):
        self.node_colors = node_colors
    
    def process_color(self, units, index):
        color = self.node_colors[index]
        RGB = color.RGB
        return "rgba(%d, %d, %d, 1.0)" %tuple(RGB)
    
class OilHandle(NodeHandle):
    __slots__ = ()
    
    def stream_width(self, stream): 
        value = stream.imass['Oil'] + stream.imass['Biodiesel']
        percent = 100 * value / oc.oilcane.imass['Oil']
        return value if percent > 0.1 else 0.
    def stream_color(self, stream): return "rgba(%d, %d, %d, 1.0)" %tuple(other.RGB)
    link_color = stream_color    
    
@bst.utils.piping.ignore_docking_warnings
def create_sankey_plot(name):
    oc.load(name, reduce_chemicals=False)
    if name == 'O1':
        loaded = getattr(oc, 'loaded_1', False)
        feedstock_handling, *sankey_groups, CHP, utilities, hxn, storage = oc.unit_groups
#         oc.pressed_bagasse.ID = 'bagasse'
    elif name == 'O2':
        loaded = getattr(oc, 'loaded_2', False)
        feedstock_handling, *sankey_groups, wastewater, oil_ext, CHP, biod_prod, utilities, hxn, storage = oc.unit_groups
        sankey_groups += [oil_ext, biod_prod]
    else:
        raise ValueError('name is invalid')
    if not loaded:
        oc.shredded_cane.ID = 'oilcane'
        oc.rvf_wash_water.ID = 'RVF_wash_water'
        for i in storage.units: 
            if 'storage' in i.line.lower():
                i.replace_with()
        if name == 'O1':
            for i in oc.M801.ins: i.ID = 'process_water'
            oc.P304.outs[0].ID = 'process_water'
            oc.R601.outs[0].ID = 'N2_purge'
            oc.loaded_1 = True
        elif name == 'O2':
            oc.loaded_2 = True
            oc.evaporator_condensate._ID = 'process_water'
            oc.pretreatment_wastewater.ID = 'wastewater'
            oc.s55.ID = oc.s46.ID ='process_water'
            oc.s108.ID = 'wastewater'
            oc.s78.ID = 'wastewater'
            oc.s104.ID = 'N2_purge'
    ignored = []
    sankey_installed_cost = {u.name: sum([i.installed_cost for i in u.units]) for u in sankey_groups}
    max_installed_cost_of_groups = max(sankey_installed_cost.values())
    oilhandle = OilHandle(
        max_feeds=100, max_products=100,
        ignore=ignored,
        node_colors=[oc.area_colors[i.name] for i in sankey_groups],
    )
    nodes = [] 
    nodes_dct = {i.name: i for i in nodes}
    dct = globals()
    dct.update(locals())
    nodes = oilhandle.nodes(sankey_groups) # + waterhandle.nodes(sankey_groups)
    links = sum([i.links() for i in nodes], [])
    if name == 'O1':
        columns = [
            ['oilcane', 'imbibition water', 'lime', 'RVF wash water', 'polymer', 'H3PO4',],
            ['Juicing', 'stripping water', 'denaturant'],
            ['filter cake', 'bagasse', 'fiber fines', 'EtOH prod.', 'catalyst', 'methanol', 'pure glycerine', 'biodiesel wash water', 'NaOH', 'HCl', 'N2'],
            ['ethanol', 'vent', 'Yeast', 'Oil ext.'],
            ['Biod. prod.', 'process water', 'vinasse', 'N2 purge', 'wastewater', 'polar lipids', 'biodiesel', 'crude glycerol'],
        ]
    data = sankey.sankey_data(nodes, links=links, orientation="v")
    fig = sankey.sankey_figure(nodes, links=links, orientation="v", add_label=False)
    fig.update_layout(font_size=10, width=250, height=600)
    fig.show()
    dct = globals()
    dct.update(locals())

Using matplotlib backend: <object object at 0x000001F29C49BA30>


In [10]:
create_sankey_plot('O2')

In [20]:
feeds = [i for i in links if isinstance(i.source, sankey.FeedNode)]
products = [i for i in links if isinstance(i.sink, sankey.ProductNode)]
total = sum([i.value() for i in feeds])
for i in products: print(i, f"{i.value()/ total:.1%}")
print(sum([i.value() for i in products]) / total)

<Link: Biod. prod. - biodiesel> 75.3%
<Link: Biod. prod. - s110> 0.5%
<Link: EtOH prod. - lignin> 10.3%
<Link: Juicing - bagasse to boiler> 5.7%
<Link: Biod. prod. - polar lipids> 8.3%
<Link: Oil ext. - wastewater> 0.8%
1.0093952744419876


In [21]:
create_sankey_plot('O1')

In [22]:
feeds = [i for i in links if isinstance(i.source, sankey.FeedNode)]
products = [i for i in links if isinstance(i.sink, sankey.ProductNode)]
total = sum([i.value() for i in feeds])
for i in products: print(i, f"{i.value()/ total:.2%}")
print(sum([i.value() for i in products]) / total)

<Link: Biod. prod. - biodiesel> 53.80%
<Link: Biod. prod. - polar lipids> 5.93%
<Link: Oil ext. - vinasse> 0.58%
<Link: Biod. prod. - wastewater> 0.34%
<Link: Juicing - bagasse> 40.00%
1.006563900294459


In [3]:
oc.process_water

AttributeError: module 'biorefineries.oilcane' has no attribute 'process_water'

In [ ]:
data = sankey.sankey_data(nodes, links=links)
fig = sankey.sankey_figure(nodes, links=links)
print(4)
fig.show()

In [ ]:
oilnodes[0].links()[0].sink

In [ ]:
nodes[0].

In [ ]:
create_sankey_plot('O2')

In [ ]:
handle.stream_colorbar(orientation='horizontal')